In [1]:
import pandas as pd
import os

In [2]:
# Caminho para ler os dados brutos
data_raw_path = '../data_raw/'
# Caminho para salvar os dados processados
processed_data_path = '../data_processed/'


Carregando Todos os DataFrames

In [3]:
try:
    path_desemprego = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
    path_gtrends = os.path.join(data_raw_path, 'gtrends.csv')
    path_icc = os.path.join(data_raw_path, 'icc_consumidor_mensal.csv')
    path_ipca = os.path.join(data_raw_path, 'ipca_mensal.csv')
    path_pib = os.path.join(data_raw_path, 'pib_mensal.csv')
    path_selic = os.path.join(data_raw_path, 'selic_meta.csv')
    path_pmc = os.path.join(data_raw_path, 'pmc.csv')

    df_desemprego = pd.read_csv(path_desemprego)
    df_gtrends = pd.read_csv(path_gtrends)
    df_icc = pd.read_csv(path_icc)
    df_ipca = pd.read_csv(path_ipca)
    df_pib = pd.read_csv(path_pib)
    df_selic = pd.read_csv(path_selic)
    df_pmc = pd.read_csv(path_pmc)

except FileNotFoundError as e:
    print(f"ERRO: Arquivo não encontrado: {e.filename}")
except Exception as e:
    print(f"ERRO ao carregar DataFrames: {e}")

Transformação consolidada das colunas data

In [4]:
# Transformação df_desemprego
codigos_str = df_desemprego['periodo_codigo_trimestre'].astype(str) # Converte o int64 para string
anos = codigos_str.str[:4] # Extrai o ano (primeiros 4) e o trimestre (último)
trimestres_num = codigos_str.str[-1]  # Pega apenas o último dígito para o trimestre
periodos_q_str = anos + "Q" + trimestres_num # Cria uma string no formato "AAAAQT"
periodos_index = pd.PeriodIndex(periodos_q_str, freq='Q') # Converte essa string para um objeto PeriodIndex trimestral
datas_fim_trimestre = periodos_index.to_timestamp(how='end') # Converte o PeriodIndex para Timestamps no FIM do período
df_desemprego['data'] = datas_fim_trimestre # Cria a nova coluna 'data' e remove a antiga
df_desemprego = df_desemprego.drop(columns=['periodo_codigo_trimestre']) #Exclui a coluna antiga


# Definir os DataFrames e seus formatos esperados para coluna date
dataframes_para_converter = [
    {"nome": "Selic",        "df": df_selic,      "formato": "%d/%m/%Y"},
    {"nome": "IPCA",         "df": df_ipca,       "formato": "%d/%m/%Y"},
    {"nome": "PIB (IBC-Br)", "df": df_pib,        "formato": "%d/%m/%Y"},
    {"nome": "gtrends",      "df": df_gtrends,    "formato": "%Y-%m-%d"},
    {"nome": "ICC",          "df": df_icc,        "formato": "%d/%m/%Y"},
    {"nome": "pmc",          "df": df_pmc,        "formato": "%Y-%m-%d"}
]

for item in dataframes_para_converter:
    nome_df = item["nome"]
    df = item["df"]
    formato_data = item["formato"]

    # Verifica se já não é datetime
    if not pd.api.types.is_datetime64_any_dtype(df['data']):
        try:
            # Converte usando o formato especificado
            df['data'] = pd.to_datetime(df['data'], format=formato_data)
            print(f"   > Coluna 'data' do DataFrame '{nome_df}' convertida.")
        except Exception as e:
            formato_tentado = formato_data if formato_data else "automático"
            print(f"   ERRO ao converter 'data' no DataFrame '{nome_df}' (formato tentado: {formato_tentado}): {e}")

   > Coluna 'data' do DataFrame 'Selic' convertida.
   > Coluna 'data' do DataFrame 'IPCA' convertida.
   > Coluna 'data' do DataFrame 'PIB (IBC-Br)' convertida.
   > Coluna 'data' do DataFrame 'gtrends' convertida.
   > Coluna 'data' do DataFrame 'ICC' convertida.
   > Coluna 'data' do DataFrame 'pmc' convertida.


Colocando colunas 'data' como indexadores

In [5]:
dfs_para_indexar = {
    "Selic": df_selic,
    "IPCA": df_ipca,
    "PIB (IBC-Br)": df_pib,
    "ICC": df_icc,
    "Desemprego": df_desemprego,
    "Gtrends": df_gtrends,
    "pmc": df_pmc
}

for nome, df in dfs_para_indexar.items():
    if 'data' in df.columns:
        try:
            df.set_index('data', inplace=True)
            df.sort_index(inplace=True)
            print(f"   > Índice definido e ordenado para DataFrame '{nome}'.")
            
        except KeyError:
             print(f"   AVISO: Coluna 'data' não encontrada em '{nome}' (talvez já seja índice?).")
        except Exception as e:
            print(f"   ERRO ao definir índice para '{nome}': {e}")


   > Índice definido e ordenado para DataFrame 'Selic'.
   > Índice definido e ordenado para DataFrame 'IPCA'.
   > Índice definido e ordenado para DataFrame 'PIB (IBC-Br)'.
   > Índice definido e ordenado para DataFrame 'ICC'.
   > Índice definido e ordenado para DataFrame 'Desemprego'.
   > Índice definido e ordenado para DataFrame 'Gtrends'.
   > Índice definido e ordenado para DataFrame 'pmc'.


Verificação dos Dataframes Finais (ajustados)

In [6]:
dataframes = {
    "Desemprego": df_desemprego,
    "Google Trends": df_gtrends,
    "ICC Consumidor": df_icc,
    "IPCA Mensal": df_ipca,
    "PIB": df_pib,
    "Selic Meta": df_selic,
    "pmc": df_pmc
}

files_list = os.listdir(data_raw_path)

for nome, df in dataframes.items():
    print(f"\nAnalisando {nome}")
    try:
        df.info()
        null_counts = df.isnull().sum()
        print(null_counts[null_counts > 0])
        if null_counts.sum() == 0:
            print("Nenhum valor ausente encontrado\n")
        
        print(df.tail())
    
    except Exception as e:
        print(f"ERRO ao inspecionar o DataFrame {nome}: {e}")



Analisando Desemprego
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23 entries, 2019-12-31 23:59:59.999999999 to 2025-06-30 23:59:59.999999999
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   taxa_desocupacao  23 non-null     float64
dtypes: float64(1)
memory usage: 368.0 bytes
Series([], dtype: int64)
Nenhum valor ausente encontrado

                               taxa_desocupacao
data                                           
2024-06-30 23:59:59.999999999               6.9
2024-09-30 23:59:59.999999999               6.4
2024-12-31 23:59:59.999999999               6.2
2025-03-31 23:59:59.999999999               7.0
2025-06-30 23:59:59.999999999               5.8

Analisando Google Trends
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70 entries, 2020-01-01 to 2025-10-01
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0 

Agregando indicadores para granularidade Mensal ('ME')

In [7]:
try:
    df_selic_mensal = df_selic.resample('ME').mean()
    df_gtrends_mensal = df_gtrends.resample('ME').mean()
    # .ffill() (forward fill) pega o valor do trimestre e o repete para os meses seguintes, até encontrar o próximo valor trimestral.
    df_desemprego_mensal = df_desemprego.resample('ME').bfill().ffill()
    df_icc_mensal = df_icc.resample('ME').last()
    df_pmc_mensal = df_pmc.resample('ME').last()
    df_ipca_mensal = df_ipca.resample('ME').last()


except Exception as e:
    print(f"ERRO durante a agregação mensal: {e}")

Concatenando Dataframes Mensais

In [8]:
lista_dfs_mensais = [
        df_selic_mensal,
        df_pmc_mensal,
        df_gtrends_mensal,
        df_ipca_mensal,
        df_icc_mensal,
        df_desemprego_mensal
    ]

df_consolidado = pd.concat(lista_dfs_mensais, axis=1)

novos_nomes = {
        'selic_meta': 'selic_media_mensal',
        'indice_PMC': 'pmc_vendas_varejo',
        'ipca_mensal': 'ipca_variacao_mensal',
        'icc_consumidor_mensal': 'icc_confianca_consumidor',
        'taxa_desocupacao': 'desemprego_taxa',
        'periodo_codigo_trimestre': 'periodo_mensal'
    }

for empresa in df_gtrends_mensal.columns:
    if empresa not in novos_nomes:
        novo_nome = f'gtrends_{empresa.lower().replace(' ', '_')}'
        novos_nomes[empresa] = novo_nome


df_master = df_consolidado.rename(columns=novos_nomes)

df_master_final = df_master.dropna()

Salvar o dataframe final (Data_processed)

In [9]:
try:
    if 'processed_data_path' not in locals():

        processed_data_path = os.path('..', 'data_processed')
        os.makedirs(processed_data_path, exist_ok=True)

    caminho_final = os.path.join(processed_data_path,'indicadores_mensal.csv')
    df_master_final.to_csv(caminho_final, index=True)


    print(f"\n   > Sucesso! DataFrame final salvo em:{caminho_final}")

except Exception as e:
    print(f"ERRO ao salvar o arquivo final: {e}")


   > Sucesso! DataFrame final salvo em:../data_processed/indicadores_mensal.csv
